In [57]:
# import libraries
import pandas as pd
import numpy as np 
from SOM_plus_clustering.modules.som import SOM
from SOM_plus_clustering.modules.kmeans import KMeans
from SOM_plus_clustering.modules.variables import INITIATION_METHOD_LIST
from sklearn import preprocessing
from sklearn.metrics import silhouette_score
from tqdm import tqdm
import time

ImportError: cannot import name 'bcubed_recall' from 'SOM_plus_clustering.modules.evals' (c:\Users\Evint\Documents\Projects\SOM_Performance_Analysis\SOM_plus_clustering\modules\evals.py)

In [49]:
# read dataset to be tested in clustering
df = pd.read_csv('iris_data.csv', header=None)
df.drop(4, axis=1, inplace=True)
df.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [50]:
def get_sc(X,y):
    try:
        return silhouette_score(X,y)
    except:
        return -1

In [53]:
def implicit_test_clustering_method(X: np.ndarray, total_trial: int, som_max_iteration:int, som_lr: float, som_nr:int, epoch:int, kmeans_total_cluster:int, som_m:int, som_n:int, path:str) -> pd.DataFrame:
    """
    Collects silhouette score of kmeans and SOM method with different initiator method.

    Args:
        X (np.ndarray): Values of test dataset.
        total_trial (int): Number of iteration for each clustering.
        som_max_iteration (int): Maximum iteration for self organizing matrix.
        som_lr (float): Value of learning rate (alpha) of Self Organzing Matrix.
        som_nr (int): Value of radius (gamma) of Self Organizing Matrix.
        epoch (int): Number of training iteration.
        kmeans_total_cluster (int): Total number of center that initiated in kmeans.
        som_m (int): Height of matrix in Self Organizing Matrix.
        som_n (int): Width of matrix in Self Organizing Matrix.

    Returns:
        pd.DataFrame: Table of silhouette score for each method.
    """
    list_sil_data = []
    list_ex_data = []
    list_time = []
    for i in tqdm(range(total_trial)):
        list_ex_time = []
        kmeans_model2 = KMeans(n_clusters=kmeans_total_cluster, method="kmeans++")
        ts = time.time()
        kmeans_model2.fit(X, epochs=epoch)
        ts = time.time() - ts
        list_ex_time.append(ts)
        kmeans_model2_pred = kmeans_model2.predict(X)
        list_kmeans_eval = [get_sc(X, kmeans_model2_pred)]
        list_som_model_eval = []
        
        for methods in INITIATION_METHOD_LIST:
            if methods != "kde_kmeans":
                som_model = SOM(m = som_m, n = som_n, 
                                dim = X.shape[1], initiate_method = methods, 
                                max_iter = som_max_iteration, 
                                learning_rate = som_lr, neighbour_rad = som_nr, distance_function="euclidean")
                ts = time.time()
                som_model.fit(X=X, epoch=epoch)
                ts = time.time() - ts
                list_ex_time.append(ts)
                pred = som_model.predict(X=X)
                #eval_score = som_model.evaluate(X=X, method="silhouette")
                eval_score = get_sc(X, pred)
                list_som_model_eval.append(eval_score)
        print(list_kmeans_eval+list_som_model_eval)
        list_sil_data.append(list_kmeans_eval+list_som_model_eval)
        list_time.append(list_ex_time)
        print("saving data") 
        data_table = pd.DataFrame(list_sil_data, columns=["kmeans++", "random SOM", "kmeans SOM", "kmeans++ SOM", "SOM++", "kde SOM"])
        time_table = pd.DataFrame(list_time, columns=["kmeans++", "random SOM", "kmeans SOM", "kmeans++ SOM", "SOM++", "kde SOM"])
        data_table.to_csv(path, index=False)
        time_table.to_csv("Datas/time_execution.csv", index=False)
    return
        

In [54]:
# extract value from dataframe
test_values = df.values

# normalize the data
test_values = preprocessing.normalize(test_values)

# create a table of silhouette score
implicit_test_clustering_method(X = test_values, 
                                total_trial = 50, 
                                som_max_iteration = None, 
                                som_lr = 0.7, 
                                som_nr = 4, 
                                epoch = 10000, 
                                kmeans_total_cluster = 8, 
                                som_m = 4, 
                                som_n = 2,
                                path="Datas/silhouette_score_data_dummy.csv")

  2%|▏         | 1/50 [03:49<3:07:26, 229.52s/it]

[0.31630607170422614, 0.3436710273057093, -1, 0.5304224254143316, 0.4363819407007648, 0.5988408665334525]
saving data


  4%|▍         | 2/50 [07:38<3:03:17, 229.11s/it]

[0.2773043669089285, 0.8022198818167943, -1, -0.10456579137080574, 0.2825636175677777, 0.38279482394302383]
saving data


  6%|▌         | 3/50 [11:28<2:59:50, 229.59s/it]

...

[0.2753849543800475, 0.46865404383891646, -1, 0.2835128930776285, 0.6074274952041135, 0.4181682867679358]
saving data


 98%|█████████▊| 49/50 [3:01:29<03:57, 237.77s/it]

[0.2733531924665259, 0.39836458513750983, -1, 0.425365076575279, 0.34035490838829147, 0.5772898282302897]
saving data


100%|██████████| 50/50 [3:05:49<00:00, 222.99s/it]

[0.2763724804427779, 0.29355584439346616, -1, 0.3796923116433043, 0.14429618224780233, 0.6110384975539538]
saving data


In [ ]:
# explicit test
